In [ ]:
# -*- coding: utf-8 -*-

 

### Import START ###

import numpy as np
import pandas as pd
import os
import glob
import pyodbc
import socket
import datetime
import time
import pickle
import sqlalchemy as sa
import urllib
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Float, Date, DateTime
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys

from Connect_Azure_Blob import *
 

### Test Env START###

print('Test Environment')

try:

    'env.yml' in sys.prefix

    print(sys.prefix)

except:

    print('Environment not set with yml file')

 
### Functions & Classes START - Ordered according to Work Flow ###

 

Base = declarative_base()       # from sqlalchemy

connect_str = get_conn_str()

def set_blob_connections(get_conn_str):


    ''' This function makes connections to different blobs on Azure Storage

        Given the hidden tokens, this code is system agnostic

    '''
   
    # blob containing opp & opp_lines files in SFDC History/:

    opp_container_client = get_container_client(connect_str, 'sfdcsnapshots')

    try_connection_azure_blob(opp_container_client)

   

    # blob containing where flat files of corresponding opp & opp_lines are saved: ### produce it ###

    flatfiles_container_client = get_container_client(connect_str, 'flatfiles')

    try_connection_azure_blob(flatfiles_container_client)

 

    # blob containing cached files:

    cache_container_client = get_container_client(connect_str, 'cache')

    try_connection_azure_blob(cache_container_client)

 

    # blob containing Live_Summary_Table_PreAgg.csv files:  ### produce file ###

    summary_table_container_client = get_container_client(connect_str, 'summarytable')

    try_connection_azure_blob(summary_table_container_client)

 

   # log_blob:

    Error_Logs_container_client = get_container_client(connect_str, 'errorlogs')

    try_connection_azure_blob(Error_Logs_container_client)

 

    # blob containing the final files:                     ### produce file ###

    final_file_container_client = get_container_client(connect_str, 'finalfile')

    try_connection_azure_blob(final_file_container_client)

 

    return opp_container_client, flatfiles_container_client, cache_container_client, summary_table_container_client, Error_Logs_container_client, final_file_container_client

 

def load_file_lists(opp_container_client, flatfiles_container_client ,mode='recent'):

 

    ''' pull and stores Opportunities and Opp_lines files' names into a list of files to process

        given correspoding (same date in file name) flat file's names that were already processed  '''


    prev_files = [file.name for file in flatfiles_container_client.list_blobs() if 'Flat_Opp_and_Lines_' in file.name] # glob.glob(r'*Flat_Opp_and_Lines_*')

    print(len(prev_files))  ### Test - REMOVE IN FINAL VERSION ###


    opps = [file.name for file in opp_container_client.list_blobs() if 'opportunities' in file.name]

    lines = [file.name for file in opp_container_client.list_blobs() if 'line_items' in file.name]

 
    date_list_opps = [file_name[13:21] for file_name in opps]

    date_list_lines = [file_name[13:21] for file_name in lines]

 

    assert set(date_list_lines)==set(date_list_opps), 'misaligned dates in source files'

 

    date_file_struct = []   # list of dictonaries with keys: 'timestamp', 'opp_file' and 'line_file'

 

    for dt in date_list_opps:

        file_exists_flag = len([file_name for file_name in prev_files if dt in file_name])>0

        if ((mode=='recent') and (file_exists_flag==False)) or (mode!='recent'):

            temp_dict = {'timestamp':dt,

                            'opp_file':[file_name for file_name in opps if dt in file_name][0],

                            'line_file':[file_name for file_name in lines if dt in file_name][0]}

            date_file_struct.append(temp_dict)

    return date_file_struct

 

### Main Body of Code START ###

if __name__=='__main__':

 

    t0 = time.time()    # Run start tim

  

    # Load file blob clients

    opp_container_client, flatfiles_container_client, cache_container_client, summary_table_container_client, Error_Logs_container_client, final_file_container_client = set_blob_connections(get_conn_str)

 

   # Load list of files to process:

   files_to_process = load_file_lists(opp_container_client, flatfiles_container_client, mode=flat_file_calc_mode)

 

print("Script took: " + str((time.time()-t0)/60.0) + ' minutes')